In [41]:
import matplotlib.pyplot as plt
import numpy as np
import math
import scipy as sp
from scipy import signal
from lea_utils import APRBS
path_subrotinas="/home/taniel/Desktop/Projetos_Deep/BCS/subrotinas.py"
#exec(compile(open('lea_utils/param_LEA.py', "rb").read(), 'lea_utils/param_LEA.py', 'exec')) #% Roda arquivo com parâmetros do modelo BCS
#exec(compile(open(path_subrotinas, "rb").read(), path_subrotinas, 'exec'))
# exec(compile(open('exp_LEA.py', "rb").read(), 'exp_LEA.py', 'exec')) # Carrega função para leitura de dados experimentais
%matplotlib tk

Definindo as variáveis simbólicas

In [42]:

# =========================================================================
#  Define as entradas do BCS LEA   
# =========================================================================
#   freq = Cexp.referencia_frequencia_inversor(1:nsim)*0.1;                   % [Hz] frequencia de operacao
# intervalo=np.array([int(1.5*3600),int(2.5*3600)])
# LEA=dados_LEA_Exp("/home/taniel/Desktop/Projetos_Deep/BCS/Simulador_BCS_LEA/Dados_BCSLEA_20210818.mat",intervalo)
# pman=LEA['pressao_manifold_coriolis']*1e5
# pres=LEA['pressao_reservatorio']*1e5
# #Entradas aplicadas no LEA
# val_pneumatica = LEA['valvula_pneumatica_topo']#[0:nsim]
# frequencia = LEA['referencia_frequencia_inversor']#[0:nsim]
# pm_0=pman[0]
# pr_0=pres[0]
# u_0=np.array([LEA['referencia_frequencia_inversor'][0],LEA['valvula_pneumatica_topo'][0],pm_0,pr_0])
# x_0=np.array([LEA['pressao_fundo'][0]*1e5,LEA['pressao_choke'][0]*1e5,LEA['vazao'][0]/3600])
# u_0
# nsim=LEA['tempo']+1
# ts=LEA['Ts']
# tempo_hora = np.arange(0,nsim*ts,ts)/3600

# plot_exogenous(tempo_hora,[LEA['referencia_frequencia_inversor'],LEA['valvula_pneumatica_topo'],pman,pres])


In [43]:
# Entradas
# Valores máximos e mínimos para normalização
#Entradas - conforme binder e pavlov
#========================================
#========================================
# Limites das variáveis para normalização LEA
#========================================
flim=[35,65]
zlim=[5,70]
pbhlim=(1e5,2.5e5) 
pwhlim=(2e5,11e5) 
qlim=(0.1/3600,5/3600)
def Lim_c(x):
    return x[1]-x[0]
pbc=Lim_c(pbhlim)
pwc=Lim_c(pwhlim)
qc=Lim_c(qlim)
pbmin=pbhlim[0]
pwmin=pwhlim[0]
qmin=qlim[0]
xss = np.float32(np.array([8311024.82175957,2990109.06207437,0.00995042241351780]))
x0=np.array([pbmin,pwmin,qmin])#,0,0])
xc=np.array([pbc,pwc,qc])#,1,1])
xssn = (xss-x0)/xc
xssn

array([54.74016667,  3.10012111,  7.29010646])

In [44]:
# Criando simbolica
from casadi import MX,vertcat,Function
from casadi import vertcat as csvertcat
from casadi import sqrt as csqrt
from casadi import fabs,sum1,nlpsol,integrator
nx = 3; nu = 4;
x = MX.sym("x",nx); # Estados
u = MX.sym("u",nu); # Exogena
dudt_max = MX.sym("dudt_max",2); # Exogena
from lea_utils import mod
from lea_utils import model as ED
ts=100
dfq_max = 0.5;    # m�xima varia��o em f/s
dzc_max = 1;      # máxima variação em zc
dx1,dx2,dx3,pin,H=ED.EDO(x,u)


In [45]:
dxdt = csvertcat(dx1,dx2,dx3) 
# dxdt = casadi.vertcat(dpbhdt,dpwhdt,dqdt) 
Eq_Estado = Function('Eq_Estado',[x,u],[dxdt],
                     ['x','u'],['dxdt'])

In [46]:
y=csvertcat(pin,H);
ny = y.size1()
# Equações algébricas
sea_nl = Function('sea_nl',[x,u],[y,pin,H],\
                 ['x','u'],['y','pin','H']); # Sistema de Eq. Algebricas variaveis de sa�da

BCS={
     'x': x,
     'u': u,
     'y': y,
     'nx': nx,
     'nu': nu,
     'ny': ny,
     'NaoLinear': {'sedo_nl': Eq_Estado(x,u),
                   'sea_nl': sea_nl
                   }
}
#%% Calculo do estacionario
#% Func��o objetivo
dxdt_0 = Eq_Estado(BCS['x'], BCS['u']);
J = sum1(dxdt_0**2);
#% Otimizador
opt={
     'ipopt':{
         'print_level':0,
         'acceptable_tol':1e-8,
         'acceptable_obj_change_tol':1e-6,
         'max_iter':150
         },
     'print_time':0,
     }

opt['ipopt']['print_level']=0;# %0,3
opt['print_time']=0;
opt['ipopt']['acceptable_tol']=1e-8;
opt['ipopt']['acceptable_obj_change_tol']=1e-6;
opt['ipopt']['max_iter']=50;


In [47]:
MMQ = {'x':BCS['x'], 'f':J, 'p':BCS['u']}
#nlp={'x':vertcat(BCS['x'],BCS['u']), 'f':J} #variáveis de decisão, função f, g (N/A)
#nlp={'x':BCS['x'], 'f':J}
solver = nlpsol('solver', 'ipopt', MMQ, opt)
# Restrições das variaveis de decis�o
# minimo
args={
      'lbx': np.zeros((nx,1)),
# m�ximo
      'ubx':np.full((nx, 1), np.inf)
      }

# Solu��o do otimizador
sol=solver(x0=BCS['x'], lbx=args['lbx'], ubx=args['ubx'], p=BCS['u']);
yss=sea_nl(sol['x'],BCS['u'])
Estacionario = Function('Estacionario',[BCS['x'],BCS['u']],[sol['x']],['x0','uss'],['xss']);


In [48]:
BCS['Estacionario'] = Estacionario;
uss=np.array([6.0000000e+01, 2.6001341e+01, 1.3833094e+04, 1.9062502e+05])
Estacionario(xssn,uss)*xc+x0


DM([420858, 2.77493e+09, 0.00868571])

In [49]:
# Definir variaveis manipuladas e controladas e disturbio externo
mv = [0,1]    #% [f, Zc]
pv = [0,1]  #% [pin, H]  #% [P, I]
#pv = [2,3];  #% [pin, H]
de = 2;      #% [pm]
tg = 2;      #% MV target
#% Parametros

#nsim=286
#%Modelo de predição
#% Criando o objeto para predição do modelo
# Iniciando variavel dicionário para a construção da EDO
# sedo = {'x': BCS['x'][0:3], # Estados
#         'p': BCS['u'], #Variáveis exogenas
#         'ode': BCS['NaoLinear']['sedo_nl'] # SEDO (Gerado no bcs_settings)
#         };

sedo = {'x': BCS['x'], # Estados
        'p': BCS['u'], #Variáveis exogenas
        'ode': Eq_Estado(x,u) # SEDO (Gerado no bcs_settings)
        };

#% Criando o objeto p,ra integração da Eq_estado
opt = {'tf':ts,
       't0':0

       };   #% opções do integrador

In [50]:
int_odes = integrator('int_odes','cvodes',sedo,opt);
# objeto integrador
res = int_odes(x0=BCS['x'],p=BCS['u']);             #   % solução um passo a frente
npv = len(pv); nmv = len(mv);


In [51]:
# Criando o objeto para solução da equação de medição
Eq_medicao = Function('Eq_medicao',[BCS['x'],BCS['u']],[BCS['y'][pv]],['x','u'],['y']);
# Criacao do objeto para simulacao do BCS Eq de estado + Eq de Medicao
Modelo_Predicao = Function('Modelo_Predicao',[BCS['x'],BCS['u']],[res['xf'],Eq_medicao(res['xf'],BCS['u'])],['xk_1','uk_1'],['xk','yk']);
Modelo_Predicao2 = Function('Modelo_Predicao2',[BCS['x'],BCS['u']],[res['xf']],['xk_1','uk_1'],['xk']);

In [52]:
# Yk = np.zeros((npv,1))
# Uk = np.zeros((nmv,1))
# Ymk = Yk
# Ys = Yk
# Ymin = Yk
# Ymax = Yk

In [53]:
#Valores iniciais de simulação
tss=5 # Tempo inicial no estacionário

# Povoar vetor de X com dados não normalizados



In [54]:
def degrau(valor,time_step,time_vector):
    v=np.zeros_like(time_vector)
    for i in np.arange(0,len(time_vector)):
        if time_vector[i]<time_step:
            v[i]=valor[0]
        else:
            v[i]=valor[1]

    return v
# time points

time = np.arange(0.5, 1.8*3600, ts)
zc2=degrau([25,42],1.5*3600,time)# exogenous input
pm2=degrau([0.12,0.12],1*3600,time)*1e5# exogenous input
f2=degrau([60,56.5],0.8*3600,time)# exogenous input
pr2=degrau([1.9,1.7],1.1*3600,time)*1e5# exogenous input
2.5*3600/ts


tempo_hora=time
time.shape
ts



100

In [55]:
# fig3=plt.figure()
# label = ['Pman','Pr','f','z'];
# ax1=fig3.add_subplot(2,2,1)
# #ax1.plot(tempo_hora ,pman/1e5, label='Pman')
# #ax1.plot(tempo_hora ,output_signal/1e5, ':r')
# ax1.plot(tempo_hora ,pm2/1e5, '--k')
# ax1.set_ylabel(label[0])
# plt.grid(True)
# ax2=fig3.add_subplot(2,2,3)
# ax2.plot(tempo_hora,pr2/1e5, '--k')
# #ax2.plot(tempo_hora,pres/1e5, label='Pr')
# ax2.set_ylabel(label[1])

# plt.grid(True)
# ax3=fig3.add_subplot(2,2,2)
# #ax3.plot(tempo_hora,LEA['referencia_frequencia_inversor'], label='f')
# ax3.plot(tempo_hora,f2, '--k')
# ax3.set_ylabel(label[2])
# plt.grid(True)
# ax4=fig3.add_subplot(2,2,4)
# #ax4.plot(tempo_hora,LEA['valvula_pneumatica_topo'], label='z')
# ax4.set_ylabel(label[3])
# ax4.plot(tempo_hora,zc2, '--k')
# plt.grid(True)

In [56]:
nsim=36

nstep = nsim

a_range = [35,65]
b_range = [30,50] # periodo de retenção em cada amplitude

u_f,prbs=APRBS(a_range,b_range,nstep)
u_z,prbs=APRBS(a_range,b_range,nstep)

uk_1 = np.array([np.ones_like(u_z)*40,u_z]);
#uk_1 = np.array([frequencia,val_pneumatica,pman,pres]);
prfixo=1.8e5; pr2=np.ones_like(f2)*prfixo
pmfixo=0.12e5;pm2=np.ones_like(f2)*pmfixo
zfixo=35;zc2=np.ones_like(f2)*zfixo
uk_1 = np.array([f2,zc2,pm2,pr2]);


#uk_1 = np.array([frequencia,val_pneumatica,np.ones_like(frequencia)*0.12*1e5,np.ones_like(frequencia)*1.9*1e5]);
#uk_1 = np.array([u_f,u_z,np.ones_like(frequencia)*0.12*1e5,np.ones_like(frequencia)*1.9*1e5]);
#uk_1 = np.array([np.ones_like(frequencia)*30,np.ones_like(frequencia)*30,np.ones_like(frequencia)*0.12*1e5,np.ones_like(frequencia)*1.9*1e5]);
# uk_1 = np.array([u_f,u_z]);
#Uss e xss simples
#uss=np.array([30,30,0.12*1e5,1.9*1e5]);
xss=Estacionario(xssn,uss)
xss
xss=Estacionario(xss,uss)

In [57]:
#xpk = Modelo_Predicao2(xssn,uss) #valores normalizados
uss=np.array([60,35,0.12e5,1.8e5])
uss=uss.reshape([4,1])
xss=Estacionario(xssn,uss)*xc+x0
xpk = Modelo_Predicao2(xss,uss) #valores normalizados
Xk=xpk*xc+x0
Uk= uss
xpk.shape

(3, 1)

In [58]:
print("Simulação iniciada")
teste=5
xi=tempo_hora[:teste]
teste=nsim;xi=tempo_hora
xi=time;teste=len(time)
#for k in range(1,5):
for k in range(1,teste):
    # print(uk_1[:,k])
    # print(xpk)
    xpk = Modelo_Predicao2(xpk,uk_1[:,k])
    Xk = np.concatenate([Xk,xpk*xc+x0],axis=1) #desnormalizar x e preencher vetor
    #ypk=sea_nl(xpk,uk_1[:,k])[0]
    #Yk = hcat([Yk,ypk*np.array([1,Hc])]);
    Uk = np.concatenate([Uk,uk_1[:,k:k+1]],axis=1)

print("Xk shape ="+str(Xk.shape))
# print("Uk shape ="+str(Uk.shape))
# print("Yk shape ="+str(Uk.shape))
# print("xi shape ="+str(xi.shape))
print("Ok.simulação concluida")
Xk[0,:].shape


Simulação iniciada
Xk shape =(3, 65)
Ok.simulação concluida


(65,)

In [59]:
# fig1=plt.figure()
# label = [r'$p_{in}(bar)$','H(m)','P','I','qc','qr' ];
# for iy in range(0,npv):
#     ax = fig1.add_subplot(npv,1,iy+1)
#     #print(iy)
#     if iy == 0: # Pin
#         ax.plot(xi,(Yk[iy,:].T)/1e5, label='Medição')
#         #ax.plot(xi,Ymk[iy,:].T/1e5, label='EKF')
#         ax.set_ylabel(label[iy])
#         ax.set(xlim=(xi[0], nsim*ts))
#        # ax.set(ylim=(40,62))
#         plt.grid(True)
#     else: # H
#         ax.plot(xi,Yk[iy,:].T, label='Medição')
#         #ax.plot(xi,Ymk[iy,:].T,label='EKF')
#         ax.set_ylabel(label[iy])
#         ax.set(xlim=(xi[0], nsim*ts))
#        # ax.set(ylim=(580, 850))
#         plt.grid(True)
# #ax.plot(xi,Yk[2,:].T, label='EKF')
# ax.legend();
# ax.set_xlabel('Time (nT)')
# fig1.show()


In [60]:
# #% Restrição
umin  = np.array([35, 0]); np.transpose(umin);  # lower bounds of inputs
umax  = np.array([65, 100]); np.transpose(umax); # upper bounds of inputs
dumax = np.array([0.5, dzc_max]); np.transpose(dumax);
fig2=plt.figure()
label = ['f(Hz)',r'$z_c$(%)'];
for iu in range(0,nmv):
    ax2=fig2.add_subplot(nmv,1,iu+1)
    ax2.plot(xi,Uk[iu,:].T, label='Medição')
    ax2.plot([0,xi[-1]],[umin[iu], umin[iu]],'--r')
    ax2.plot([0,xi[-1]],[umax[iu], umax[iu]],'--r', label='Restrição')
    ax2.set_ylabel(label[iu])
    #ax2.set(xlim=(xi[0], nsim*ts))
    if iu==0:
        ax2.set(ylim=(30, 70))

    plt.grid(True)


In [61]:
# fig3=plt.figure()
# label = ['Pbh (bar)','Pwh (bar)','q(m3/s)'];
# for iu in range(0,3):
#     ax3=fig3.add_subplot(3,1,iu+1)
#     if iu==2:
#         ax3.plot(xi,(Xk[iu,:].T)*3600, label='Medição')
#         #ax3.plot([1,nsim],[umin[iu], umin[iu]],'--r')
#         #ax3.plot([1,nsim],[umax[iu], umax[iu]],'--r', label='Restrição')
#         ax3.set_ylabel(label[iu])
#         #ax3.set(xlim=(xi[0], nsim*ts))
#         # if iu==0:
#         #     #ax2.set(ylim=(30, 70))
#         #     print(iu)
#         plt.grid(True)
#     else:
#         ax3.plot(xi,Xk[iu,:].T/1e5, label='Medição')
#         #ax3.plot([1,nsim],[umin[iu], umin[iu]],'--r')
#         #ax3.plot([1,nsim],[umax[iu], umax[iu]],'--r', label='Restrição')
#         ax3.set_ylabel(label[iu])
#         #ax3.set(xlim=(xi[0], nsim*ts))
#         # if iu==0:
#         #     #ax2.set(ylim=(30, 70))
#         #     print(iu)
#         plt.grid(True)

fig3=plt.figure()
label = ['Pbh (bar)','Pwh (bar)','q(m3/h)'];
for iu in range(0,3):
    ax3=fig3.add_subplot(3,1,iu+1)
    if iu==2:
        ax3.plot(xi,(Xk[iu,:].T*3600), label='Medição')
        #ax3.plot(xi,(LEA['vazao']), ':r')
        #ax3.plot([1,nsim],[umin[iu], umin[iu]],'--r')
        #ax3.plot([1,nsim],[umax[iu], umax[iu]],'--r', label='Restrição')
        ax3.set_ylabel(label[iu])
        #ax3.set(xlim=(xi[0], nsim*ts))
        # if iu==0:
        #     #ax2.set(ylim=(30, 70))
        #     print(iu)
        plt.grid(True)
    elif iu==0:
        ax3.plot(xi,Xk[iu,:].T/1e5)
        #ax3.plot(xi,LEA['pressao_fundo'],':r', label='Medição')
        #ax3.plot([1,nsim],[umin[iu], umin[iu]],'--r')
        #ax3.plot([1,nsim],[umax[iu], umax[iu]],'--r', label='Restrição')
        ax3.set_ylabel(label[iu])
        #ax3.set(xlim=(xi[0], nsim*ts))
        # if iu==0:
        #     #ax2.set(ylim=(30, 70))
        #     print(iu)
        ax3.legend()
        plt.grid(True)
    else:
        ax3.plot(xi,Xk[iu,:].T/1e5)
        #ax3.plot(xi,LEA['pressao_choke'],':r', label='Medição')
        #ax3.plot([1,nsim],[umin[iu], umin[iu]],'--r')
        #ax3.plot([1,nsim],[umax[iu], umax[iu]],'--r', label='Restrição')
        ax3.set_ylabel(label[iu])
        #ax3.set(xlim=(xi[0], nsim*ts))
        # if iu==0:
        #     #ax2.set(ylim=(30, 70))
        #     print(iu)
        plt.grid(True)
        


No handles with labels found to put in legend.


In [62]:
# Xk[2,:]=Xk[2,:]*xc[2] #desnormalizar vazão
# Yk[1,:]=Yk[1,:]*Hc #desnormalizar Head

In [63]:
# #exec(compile(open('envelope.py', "rb").read(), 'envelope.py', 'exec')) #% Roda arquivo com parâmetros do modelo BCS
# fig4,ax4=plt.subplots()
# plt.grid(True)
# #BCS['Envelope']['fig'](ax4); # grafico do envelope
# #
# # Evolução dentro do envelope
# ax4.plot(Xk[2,0:].T*3600,Yk[1,0:].T,'--k')
# ax4.plot(Xk[2,0]*3600,Yk[1,0],'o')#,'MarkerFaceColor',[0,1,0],'MarkerEdgeColor',[0,0,0])
# ax4.plot(Xk[2,-1]*3600,Yk[1,-1],'o')#,'MarkerFaceColor',[1,0,0],'MarkerEdgeColor',[0,0,0])
# ax4.annotate('t=0',
#              xy=(float(Xk[2,0]*3600),float(Yk[1,0])),
#              xytext=(float(Xk[2,0]*3600)-5,float(Yk[1,0])+10),
#              arrowprops=dict(facecolor='green', shrink=0.01))

# ax4.annotate('t='+str(nsim),
#              xy=(float(Xk[2,-1]*3600),float(Yk[1,-1])),
#              xytext=(float(Xk[2,-1]*3600)-7,float(Yk[1,-1])+10),
#              arrowprops=dict(facecolor='red', shrink=0.01))
# plt.show()



In [64]:
#np.savez('LEA_data_train-f.npz', t=xi, x1=Xk[0,:].T,x2=Xk[1,:].T,x3=Xk[2,:].T,zc=zc2,f=f2,pm=pm2,pr=pr2)

In [65]:
def ler_dados(str):
    with open(str, 'r') as f:
        d = f.readlines()
        epocas = np.zeros(len(d));
        tau = np.zeros(len(d));
        j = 0;
        data = []
        for i in d:
            k = i.rstrip().split(" ")
            data.append([float(i) if is_float(i) else i for i in k])
            epocas[j] = float(k[0])
            tau[j] = float(k[1])
            j += 1;
    
    return tau, epocas


In [66]:
qcc/qc

NameError: name 'qcc' is not defined